In [ ]:
import pandas as pd
import numpy as np
import requests
import json
import datetime

df = pd.read_csv('rawdata.csv')
df
df_subcopy = df.loc[df['Units'] == "kWh"].copy()



def weekdayfunction(x):
    if x['DayOfWeek'] > 5:
        return 0
    else:
        return 1


def findout(x, i):
    df3_subcopy = df_subcopy.iloc[:, x:x + 12].apply(np.sum, axis=1).copy()

    df_subcopy.loc[:, df_subcopy.columns.get_loc("23:05") + i] = df3_subcopy

    # result=pd.concat([df_subcopy,df3_subcopy],axis=1)
    # x=x+12
    # print(df3_subcopy)

    if (x != df_subcopy.columns.get_loc("23:05")):
        i = i + 1
        return (findout(x + 12, i))

    else:
        return df_subcopy


findout(df_subcopy.columns.get_loc("0:05"), 1)
df_subcopy.drop(df_subcopy.iloc[:, 4:292], axis=1, inplace=True)
df_subcopy = pd.melt(df_subcopy, id_vars=['Account', 'Date', 'Channel', 'Units'], var_name='Hour', value_name='kWh')
df_subcopy['newDate'] = df_subcopy['Date']
import datetime as dt

df_subcopy['newDate'] = pd.to_datetime(df_subcopy['newDate'])
df_subcopy['Year'] = df_subcopy['newDate'].dt.year
df_subcopy['Month'] = df_subcopy['newDate'].dt.month
df_subcopy['Day'] = df_subcopy['newDate'].dt.day
df_subcopy['DayOfWeek'] = df_subcopy['newDate'].dt.weekday
df_subcopy = df_subcopy.sort_values(['Month', 'Day', 'Hour'], ascending=[True, True, True])
                  
df_subcopy['Weekday']=df_subcopy.apply(weekdayfunction, axis=1)

def replacefunc(i, j):
    df_subcopy['Hour'] = df_subcopy['Hour'].replace([i], j)
    if (i != 304 and j != 23):
        return replacefunc(i + 1, j + 1)
    else:
        return df_subcopy


replacefunc(281, 0)


def f1(x):
    if x['Hour'] > 6 and x['Hour'] < 20:
        return 1
    else:
        return 0


df_subcopy['PeakHour'] = df_subcopy.apply(f1, axis=1)

start_date = str(df_subcopy['newDate'][0])
len = df_subcopy['newDate'].__len__()
end_date = str(df_subcopy['newDate'][len - 1])

start_date = start_date.split(" ")[0]
end_date = end_date.split(" ")[0]
print(start_date)
print(end_date)
start_date = start_date.split("-")
end_date = end_date.split("-")

start_date = datetime.date(int(start_date[0]), int(start_date[1]), int(start_date[2]))
end_date = datetime.date(int(end_date[0]), int(end_date[1]), int(end_date[2]))

def get_temps():
    report = {}

    i = start_date
    diff = datetime.timedelta(days=1)
    while i <= end_date:
        date = i.strftime("%Y%m%d")
        url = "http://api.wunderground.com/api/b0398f0486550559/geolookup/history_" + date + "/q/MA/Boston.json"
        i += diff

        resp = requests.get(url)
        resp = json.loads(resp.text)
        times = resp["history"]["observations"]

        temps = {}
        for time in times:
            temps[int(time["date"]["hour"])] = time["tempi"]

        for x in range(24):
            if x not in temps:
                temps[x] = resp["history"]["dailysummary"][0]["meantempi"]
        report[date] = temps
    return report

temps = get_temps()

def get_temp(row):
    date = row['newDate']
    date = date._date_repr.replace("-", "")

    if date in temps:
        hour = int(row["Hour"])
        if hour in temps[date]:
            return temps[date][hour]

df_subcopy['temp'] = df_subcopy.apply(get_temp, axis=1)
#df_subcopy['NewkWh'] = df_subcopy['kWh'].groupby(df_subcopy['newDate']).transform('std')

#get_avg()

#print(df_subcopy)


df_subcopy=df_subcopy[['Account','newDate','kWh','Month','Day','Year','Hour','DayOfWeek','Weekday','PeakHour','temp']]
df_subcopy[((df_subcopy.kWh - df_subcopy.kWh.mean()) / df_subcopy.kWh.std()).abs() < 3]
df_subcopy
df_subcopy.to_csv('/Users/Jaya/Desktop/NEU/ADS/SampleFormat.csv')